In [41]:
import numpy as np
import pandas as pd
import random, operator, copy
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from mpl_toolkits.mplot3d import Axes3D

In [2]:
def split(data, label, num):
    idx = np.random.choice(data.shape[0], num, replace=False)
    data_train = data[idx]
    label_train = label[idx]
    data_test = data[[x for x in range(data.shape[0]) if x not in idx]]
    label_test = label[[x for x in range(data.shape[0]) if x not in idx]]
    return data_train, label_train, data_test, label_test

In [3]:
def sigmoid(x):
    return 1.0 / (1+np.exp(-x))

In [4]:
def tanh(x):
    return np.tanh(x)

In [5]:
def Gaussian(x):
    return np.exp(-x**2)

In [6]:
def ReLU(x):
    return np.where(x>0, x, 0)

In [7]:
def dropout(x, p):
    mask = (np.random.uniform(0,1,x.shape)<=p)/p
    return x*mask

In [8]:
def forward(Ws, data, label, activation = "sigmoid", dropP=1):
    # No need to add bias as the first column of the data is all 1s
    out = data
    for W in Ws:
        out = np.hstack([np.ones((out.shape[0], 1)),out])
        out = dropout(out, dropP)
        out = out.dot(W)
        if activation == 'sigmoid':
            out = sigmoid(out)
        elif activation == 'tanh':
            out = tanh(out)
        elif activation == 'Gaussian':
            out = Gaussian(out)
        elif activation == 'ReLU':
            out = ReLU(out)
    loss = 0.5*np.sum((out-label)**2)/data.shape[0]
    pred = np.argmax(out, axis=1)
    truth = np.argmax(label, axis=1)
    accuracy = np.mean(pred==truth)
    return loss, accuracy, pred

In [9]:
def normalizeFitness(fitness):
    return fitness/np.sum(fitness)

In [10]:
def sortPopulation(population, data, label, dropP):
    # Sort the population by their fitness by descending order
    # Return the list of tuples,
    # where the first is the index of individual in the original population 
    # and the second is the corresponding fitness. 
    fitness = {}
    for i in range(len(population)):
        _, fitness[i], __ = forward(population[i], data, label, 'ReLU', dropP)
    sortedResult = sorted(fitness.items(), key = operator.itemgetter(1), reverse = True)
    fitness = [fit for idx,fit in sortedResult]
    sortedPop = population[[idx for idx,fit in sortedResult]]
    return fitness, sortedPop

In [11]:
def initIndividual(NNPar):
    input_attr, hidden_layers, num_classes = NNPar
    Ws = np.array([None for x in range(len(hidden_layers)+1)])
    hidden_layers = [input_attr]+hidden_layers+[num_classes]
    for i in range(len(hidden_layers)-1):
        Ws[i] = np.random.uniform(-1,1,(hidden_layers[i]+1,hidden_layers[i+1]))
    return Ws

In [12]:
def initialPopulation(popSize, NNPar):
    # Create popSize population
    population = []
    for i in range(popSize):
        population.append(initIndividual(NNPar))
    return np.array(population)

In [13]:
def crossover(parent1, parent2, crossoverRate):
    # Crossover with the probability crossoverRate
    # Return the parent1 if no crossover occurs, or is the new individual. 
    if random.random()<crossoverRate:
        child = np.array([None for i in range(parent1.shape[0])])
        for i in range(parent1.shape[0]):
            tmp = np.array([parent1[i], parent2[i]])
            idx = np.random.randint(2, size=tmp.shape[2])
            child[i] = tmp[idx,:,range(tmp.shape[2])].T
        return child
    else:
        return copy.deepcopy(parent1)

In [14]:
def mutation(individual, mutationRate):
    # Randomly choose two city to swap with the probability mutationRate
    if random.random()<mutationRate:
        for i in range(individual.shape[0]):
            idx = np.random.randint(2, size=individual[i].shape)
            randnum = np.random.uniform(-1,1)
            individual[i][idx==1] = randnum

In [15]:
def nextGeneration(curPop, popSize, crossover_rate, mutation_rate, elitePer, fitness, data, label, dropP):
    eliteSize = int(elitePer*popSize)
    
    if fitness==None:
        p, sortedPop = sortPopulation(curPop, data, label, dropP)
    else:
        p = fitness
        sortedPop = curPop
    
    p = normalizeFitness(p)
    #preserve elite children
    elite = sortedPop[:eliteSize]
    
    children = []
    
    #crossover
    for p1 in sortedPop:
        p2 = sortedPop[np.random.choice(range(len(sortedPop)),p = p)]
        if p1 in elite:
            children.append(crossover(p1, p2, 1))
        else:
            children.append(crossover(p1, p2, crossover_rate))
    
    #mutation
    for i in range(len(children)):
        mutation(children[i], mutation_rate)
    children = np.vstack([children, elite])
    fitness, children = sortPopulation(children, data, label, dropP)
    return fitness[:popSize], children[:popSize]

In [16]:
dataset = pd.read_csv('./diabetes.txt', sep='\t').values[:,1:]
data = dataset[:,:-2]
label = dataset[:,-2:]

In [17]:
hidden_layers = [20,20, 10]

num_classes = 2 
input_attr = 8
mutation_rate = 0.1
crossover_rate = 0.9

num_generations = 800
popSize = 50
dropP=1
elitePer = 0.1

#data_train, label_train, data_test, label_test = split(data, label, 600)
kf = KFold(n_splits = 6)

print(np.mean(hist))

X, Y=np.meshgrid(first_layer,second_layer)

fig = plt.figure()
ax = Axes3D(fig)

Z = np.reshape(hist, (8,8))
print(Z)
print(np.argmax(Z))
plt.title('The performance of different ANN')
ax.plot_surface(X, Y, Z, rstride=1, cstride=1, cmap='rainbow')
plt.show()



In [63]:
hidden_layers = [10, 60]

num_classes = 2 
input_attr = 8
mutation_rate = 0.1
crossover_rate = 0.9

num_generations = 500
popSize = 50
dropP=1
elitePer = 0.1

batch_size = 200
train_fitness_hist = []
test_fitness_hist = []
for i in range(10):
    fitness = None
    best_fitness_hist = []
    X_train, y_train, X_test, y_test = split(data, label, 600)
    pop = initialPopulation(popSize, (input_attr, hidden_layers, num_classes))
    for g in range(num_generations):
        X_batch, y_batch, _, __ = split(X_train, y_train, batch_size)
        fitness, pop = nextGeneration(pop, popSize,
                                      crossover_rate,
                                      mutation_rate,
                                      elitePer, fitness,
                                      X_batch, y_batch, 1)
        best_fitness_hist.append(fitness[0])

        if (g+1)%800==0:
            print("Iteration %02d: %lf"%(g+1, best_fitness_hist[-1]))
    _, acc_tr, __=forward(pop[0], X_train, y_train, 'ReLU', 1)
    _, acc_test, __=forward(pop[0], X_test, y_test, 'ReLU', 1)
    train_fitness_hist.append(acc_tr)
    test_fitness_hist.append(acc_test)

/root/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:19: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.


In [64]:
print(test_fitness_hist)
np.mean(test_fitness_hist)

[0.7125748502994012, 0.7604790419161677, 0.7784431137724551, 0.7664670658682635, 0.7964071856287425, 0.7544910179640718, 0.7305389221556886, 0.7664670658682635, 0.7664670658682635, 0.7664670658682635]


0.759880239520958